I'm [building my own macOS Jupyter frontend](https://twitter.com/wasimlorgat/status/1611615676220817415?s=20) and writing about my experiences and learnings along the way. In order to do that, I need to be familiar with how Jupyter servers works.

This notebook is an executable playground for understanding and communicating with Jupyter servers. You can think of it as a barebones Jupyter frontend, since we'll be implementing the full lifecycle from creating a new notebook, writing code to it, and executing that code.

My approach to learning this was a combination of using Chrome dev tools to inspect network requests in Jupyter Lab, and reading the wonderful Jupyter server docs. I'll include links to the relevant docs in each section below.

Let's get started!

## Start the server

To start, ensure that you're running a Jupyter server in another process (e.g. in a terminal) by running the following command:

```sh
jupyter server
```

Then update the `url` and `token` variables below based on the terminal output. For example, it should output something like, but with a different `url` and `token`:

```
[C 2023-01-07 12:03:57.482 ServerApp]

    To access the server, open this file in a browser:
        file:///Users/seem/Library/Jupyter/runtime/jpserver-80287-open.html
    Or copy and paste one of these URLs:
        http://localhost:8890/?token=0799b0472813fd449b0ed5ee97431cc86b97045e88925063
     or http://127.0.0.1:8890/?token=0799b0472813fd449b0ed5ee97431cc86b97045e88925063
```

In [7]:
# NB: Update these based on your terminal output
url = 'http://localhost:8889/'
token = '14b5074a5c572641122bee0990554bbe2068c6ce4e236e68'

## Authenticate

In [17]:
import requests

First, we'll do a quick check that there is a server at the defined `url`:

In [80]:
requests.get(url)

<Response [200]>

Next we need to authenticate. What happens if we try to make a request to an endpoint that requires authentication, for example `GET /api/sessions`?

In [81]:
requests.get(url + 'api/contents')

<Response [403]>

It fails with `403 Forbidden`.

If we include our token in the `Authorization` header:

In [82]:
headers = {'Authorization': f'token {token}'}
requests.get(url + 'api/contents', headers=headers)

<Response [200]>

... it works!

Let's create a `requests.Session` so we don't have to keep specifying headers:

In [87]:
session = requests.Session()
session.headers.update(headers)

## Managing files

### List the contents of a directory

In [125]:
!rm ../temp/*

`GET /api/contents/<path>` lists the contents of the directory at `path`. You can think of it as `ls`:

In [126]:
session.get(url + 'api/contents').json()

{'name': '',
 'path': '',
 'last_modified': '2023-01-10T11:40:43.870966Z',
 'created': '2023-01-10T11:40:43.870966Z',
 'content': [],
 'format': 'json',
 'mimetype': None,
 'size': None,
 'writable': True,
 'type': 'directory'}

Since the directory is currently empty, `content` is an empty list.

### Create an empty notebook

`POST /api/contents/<path>` creates an empty file in the directory at `path`. You can specify the `type` of the file in the request body:

In [127]:
session.post(url + 'api/contents', json={'type': 'notebook'})

<Response [201]>

Let's confirm that the file exists with `GET /api/contents`:

In [128]:
session.get(url + 'api/contents').json()

{'name': '',
 'path': '',
 'last_modified': '2023-01-10T11:40:44.847743Z',
 'created': '2023-01-10T11:40:44.847743Z',
 'content': [{'name': 'Untitled.ipynb',
   'path': 'Untitled.ipynb',
   'last_modified': '2023-01-10T11:40:44.849035Z',
   'created': '2023-01-10T11:40:44.849035Z',
   'content': None,
   'format': None,
   'mimetype': None,
   'size': 72,
   'writable': True,
   'type': 'notebook'}],
 'format': 'json',
 'mimetype': None,
 'size': None,
 'writable': True,
 'type': 'directory'}

The response is a nested dict. The root dict refers to the root directory as before, however, `content` now contains the newly created notebook file `Untitled.ipynb`.

We can get the contents of this file using the same method but referring to the file's path i.e. `GET /api/contents/<path>`:

In [129]:
data = session.get(url + 'api/contents/Untitled.ipynb').json()
data

{'name': 'Untitled.ipynb',
 'path': 'Untitled.ipynb',
 'last_modified': '2023-01-10T11:40:44.849035Z',
 'created': '2023-01-10T11:40:44.849035Z',
 'content': {'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

We're probably most interested in `content`, which contains the JSON content of the notebook:

In [130]:
data['content']

{'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5}

### Rename a notebook

Our newly created file is still named `Untitled.ipynb`. Let's rename it to `sum.ipynb` with `PATCH /api/contents/<path>`:

In [131]:
session.patch(url + 'api/contents/Untitled.ipynb', json={'path': 'sum.ipynb'}).json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-10T11:40:44.849035Z',
 'created': '2023-01-10T11:40:46.480364Z',
 'content': None,
 'format': None,
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

Confirm that it's been renamed. `Untitled.ipynb` no longer exists:

In [132]:
session.get(url + 'api/contents/Untitled.ipynb').json()

{'message': 'No such file or directory: Untitled.ipynb', 'reason': None}

... but `sum.ipynb` does:

In [133]:
session.get(url + 'api/contents/sum.ipynb').json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-10T11:40:44.849035Z',
 'created': '2023-01-10T11:40:46.480364Z',
 'content': {'cells': [], 'metadata': {}, 'nbformat': 4, 'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 72,
 'writable': True,
 'type': 'notebook'}

::: {.callout-note}
You can also create a file with a specified name using `PUT /api/contents/<path>`, instead of letting the server find a unique named prefixed with `Untitled`.
:::

### Update a notebook's contents

Create a cell and append it to existing contents:

In [143]:
cell = {
    'cell_type': 'code',
    'id': '0',
    'metadata': {},
    'source': [
        '1 + 1\n',
    ],
    'outputs': [],
    'execution_count': 0,
}
data = session.get(url + 'api/contents/sum.ipynb').json()
data['content']['cells'].append(cell)

Update the notebook's contents using `PUT /api/contents/<path>`:

In [144]:
session.put(url + 'api/contents/sum.ipynb', json={'content': data['content'], 'type': 'notebook'})

<Response [200]>

Confirm that the notebook's been updated. Note that `last_modified` and `content` have both updated:

In [141]:
session.get(url + 'api/contents/sum.ipynb').json()

{'name': 'sum.ipynb',
 'path': 'sum.ipynb',
 'last_modified': '2023-01-10T11:40:50.112357Z',
 'created': '2023-01-10T11:40:50.112357Z',
 'content': {'cells': [{'cell_type': 'code',
    'execution_count': 0,
    'id': '0',
    'metadata': {'trusted': True},
    'outputs': [],
    'source': '1 + 1\n'}],
  'metadata': {},
  'nbformat': 4,
  'nbformat_minor': 5},
 'format': 'json',
 'mimetype': None,
 'size': 216,
 'writable': True,
 'type': 'notebook'}

## Executing code

### Start a session

### Open a WebSocket

### Send an `execute_request`

### Parse the responses

## Cleanup

### Close the WebSocket

### Close the session

### Save and close the file

### Shutdown the server